In [1]:
from pathlib import Path

from tqdm import trange

from prompts import ECL_CONTEXT_PROMPT, NAME_TO_AGENT
from run_scientific_meeting import run_scientific_meeting
from utils import load_summaries

In [2]:
# Set up the team
team_lead = "Principal Investigator"
team_members = tuple(NAME_TO_AGENT.keys())

# Set up key parameters
num_iterations = 3
num_rounds = 3
save_dir = Path("drug_discovery")
model = "gpt-4o"
discussion_paths = []
contexts = (ECL_CONTEXT_PROMPT,)

In [3]:
PROJECT_SELECTION_PROMPT = "You are starting on a research project that is aiming to apply artificial intelligence to drug discovery. In this meeting, you need to select a specific research direction for this project. The primary considerations are: (1) the project must have high clinical value, meaning the research contributes to helping patients, (2) the project must involve the development of an artificial intelligence model, and (3) the project must use Emerald Cloud Labs (ECL) to validate the artificial intelligence model’s output, which means that any required wet lab experiments must be within the capabilities of ECL’s scientific instrumentation. Please determine a research project that meets these criteria. Please be as specific as possible in terms of the precise goal of the project and the experiments that will be run."

In [4]:
PROJECT_SELECTION_QUESTIONS = (
    "What is the specific research direction that you are proposing?",
    "What is the clinical value of this research direction?",
    "How will you develop an artificial intelligence model for this project?",
    "How will you use Emerald Cloud Labs (ECL) to validate the artificial intelligence model’s output?",
)

In [5]:
# Run project selection
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=PROJECT_SELECTION_PROMPT,
        agenda_questions=PROJECT_SELECTION_QUESTIONS,
        contexts=contexts,
        save_dir=save_dir / "project_selection",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:09<00:45,  9.05s/it]

Team Members:  33%|███▎      | 2/6 [00:17<00:34,  8.59s/it]

Team Members:  50%|█████     | 3/6 [00:26<00:26,  8.85s/it]

Team Members:  67%|██████▋   | 4/6 [00:37<00:19,  9.66s/it]

Team Members:  83%|████████▎ | 5/6 [00:55<00:12, 12.63s/it]

Team Members: 100%|██████████| 6/6 [01:08<00:00, 11.45s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:08<03:26, 68.73s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:13<01:05, 13.02s/it]

Team Members:  33%|███▎      | 2/6 [00:24<00:48, 12.14s/it]

Team Members:  50%|█████     | 3/6 [00:44<00:47, 15.85s/it]

Team Members:  67%|██████▋   | 4/6 [00:56<00:28, 14.28s/it]

Team Members:  83%|████████▎ | 5/6 [01:08<00:13, 13.31s/it]

Team Members: 100%|██████████| 6/6 [01:20<00:00, 13.38s/it]

Rounds (+ Summary Round)

Input token count: 182,533
Output token count: 13,356
Max token length: 17,670
Cost: $1.11



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:12<01:02, 12.53s/it]

Team Members:  33%|███▎      | 2/6 [00:20<00:40, 10.13s/it]

Team Members:  50%|█████     | 3/6 [00:35<00:36, 12.24s/it]

Team Members:  67%|██████▋   | 4/6 [00:50<00:26, 13.25s/it]

Team Members:  83%|████████▎ | 5/6 [01:06<00:14, 14.23s/it]

Team Members: 100%|██████████| 6/6 [01:22<00:00, 13.68s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:22<04:06, 82.11s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:15<01:19, 15.93s/it]

Team Members:  33%|███▎      | 2/6 [00:29<00:59, 14.81s/it]

Team Members:  50%|█████     | 3/6 [00:51<00:54, 18.06s/it]

Team Members:  67%|██████▋   | 4/6 [01:14<00:40, 20.02s/it]

Team Members:  83%|████████▎ | 5/6 [01:42<00:22, 22.67s/it]

Team Members: 100%|██████████| 6/6 [02:05<00:00, 20.99s/it]

Rounds (+ Summary Round

Input token count: 218,513
Output token count: 18,520
Max token length: 22,834
Cost: $1.37



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:10<00:52, 10.42s/it]

Team Members:  33%|███▎      | 2/6 [00:20<00:39,  9.93s/it]

Team Members:  50%|█████     | 3/6 [00:31<00:32, 10.71s/it]

Team Members:  67%|██████▋   | 4/6 [00:45<00:24, 12.01s/it]

Team Members:  83%|████████▎ | 5/6 [01:12<00:17, 17.21s/it]

Team Members: 100%|██████████| 6/6 [01:24<00:00, 14.11s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:24<04:13, 84.66s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:17<01:26, 17.30s/it]

Team Members:  33%|███▎      | 2/6 [00:29<00:58, 14.53s/it]

Team Members:  50%|█████     | 3/6 [00:43<00:42, 14.02s/it]

Team Members:  67%|██████▋   | 4/6 [00:55<00:26, 13.38s/it]

Team Members:  83%|████████▎ | 5/6 [01:07<00:12, 12.97s/it]

Team Members: 100%|██████████| 6/6 [01:21<00:00, 13.65s/it]

Rounds (+ Summary Round

Input token count: 192,791
Output token count: 14,481
Max token length: 18,795
Cost: $1.18


In [3]:
# Select preferred project
discussion_paths.append(save_dir / "project_selection" / "discussion_1.json")

In [4]:
# Load summaries
summaries = load_summaries(discussion_paths=discussion_paths)

In [11]:
TARGET_SELECTION_PROMPT = "In the previous meeting, you settled on a general project direction (see summary). Now, you need to make that project more precisely defined. Please select one specific disease target and one specific drug modality for this target related to your prior discussion. Remember that you are constrained by the capabilities of Emerald Cloud Labs (ECL)."

In [ ]:
TARGET_SELECTION_QUESTIONS = (
    "What is the specific disease target that you are proposing?",
    "What is the specific drug modality that you are proposing?",
)

In [ ]:
# Run target selection
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=TARGET_SELECTION_PROMPT,
        agenda_questions=TARGET_SELECTION_QUESTIONS,
        summaries=summaries,
        contexts=contexts,
        save_dir=save_dir / "target_selection",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

In [4]:
# Select preferred target
discussion_paths.append(save_dir / "target_selection" / "discussion_1.json")

In [5]:
# Load summaries
summaries = load_summaries(discussion_paths=discussion_paths)

In [12]:
DRUG_DISCOVERY_APPROACH_PROMPT = "In the previous meeting, you chose a specific disease target and drug modality (see summary). Now, you need to be more specific about the drug discovery process. Please design a specific drug discovery approach for this target and drug modality. Specify whether to design a new drug de novo or whether to modify and improve an existing but imperfect drug candidate. In either case, decide which exact properties you will optimize for in the drug that you design. Furthermore, please specify exactly what type of machine learning model you will use to accomplish this task. Decide on a specific dataset to train the model, and describe in detail how that model will be used to design new or improved drugs. If improving an existing drug, please specify which drug. Please note: an important constraint is that you only have three months and relatively limited experimental throughput. Remember that you are also constrained by the capabilities of Emerald Cloud Labs (ECL)."

In [ ]:
DRUG_DISCOVERY_APPROACH_QUESTIONS = (
    "What is the specific drug discovery approach that you are proposing?",
    "What properties will you optimize for in the drug that you design?",
    "What type of machine learning model will you use for this task?",
    "What dataset will you use to train the model?",
    "How will the model be used to design new or improved drugs?",
    "If improving an existing drug, which drug will you improve?",
)

In [6]:
# Run drug discovery approach
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=DRUG_DISCOVERY_APPROACH_PROMPT,
        agenda_questions=DRUG_DISCOVERY_APPROACH_QUESTIONS,
        summaries=summaries,
        contexts=contexts,
        save_dir=save_dir / "drug_discovery_approach",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:08<00:43,  8.69s/it]

Team Members:  33%|███▎      | 2/6 [00:17<00:35,  8.99s/it]

Team Members:  50%|█████     | 3/6 [00:30<00:32, 10.79s/it]

Team Members:  67%|██████▋   | 4/6 [00:51<00:29, 14.60s/it]

Team Members:  83%|████████▎ | 5/6 [01:06<00:14, 14.68s/it]

Team Members: 100%|██████████| 6/6 [01:21<00:00, 13.53s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:21<04:03, 81.19s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:14<01:14, 14.93s/it]

Team Members:  33%|███▎      | 2/6 [00:29<00:57, 14.43s/it]

Team Members:  50%|█████     | 3/6 [00:41<00:40, 13.38s/it]

Team Members:  67%|██████▋   | 4/6 [00:56<00:28, 14.25s/it]

Team Members:  83%|████████▎ | 5/6 [01:11<00:14, 14.43s/it]

Team Members: 100%|██████████| 6/6 [01:26<00:00, 14.39s/it]

Rounds (+ Summary Round)

Input token count: 222,962
Output token count: 15,063
Max token length: 20,223
Cost: $1.34



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:07<00:38,  7.72s/it]

Team Members:  33%|███▎      | 2/6 [00:19<00:40, 10.02s/it]

Team Members:  50%|█████     | 3/6 [00:31<00:32, 10.77s/it]

Team Members:  67%|██████▋   | 4/6 [00:44<00:24, 12.02s/it]

Team Members:  83%|████████▎ | 5/6 [01:01<00:13, 13.53s/it]

Team Members: 100%|██████████| 6/6 [01:18<00:00, 13.00s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:18<03:54, 78.02s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:15<01:16, 15.30s/it]

Team Members:  33%|███▎      | 2/6 [00:24<00:47, 11.76s/it]

Team Members:  50%|█████     | 3/6 [00:35<00:33, 11.20s/it]

Team Members:  67%|██████▋   | 4/6 [00:47<00:23, 11.86s/it]

Team Members:  83%|████████▎ | 5/6 [00:59<00:11, 11.74s/it]

Team Members: 100%|██████████| 6/6 [01:09<00:00, 11.53s/it]

Rounds (+ Summary Round

Input token count: 205,070
Output token count: 12,794
Max token length: 17,954
Cost: $1.22



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:09<00:49,  9.95s/it]

Team Members:  33%|███▎      | 2/6 [00:21<00:43, 10.82s/it]

Team Members:  50%|█████     | 3/6 [00:34<00:36, 12.03s/it]

Team Members:  67%|██████▋   | 4/6 [00:48<00:25, 12.71s/it]

Team Members:  83%|████████▎ | 5/6 [01:06<00:14, 14.43s/it]

Team Members: 100%|██████████| 6/6 [01:19<00:00, 13.31s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:19<03:59, 79.86s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:13<01:07, 13.56s/it]

Team Members:  33%|███▎      | 2/6 [00:25<00:50, 12.68s/it]

Team Members:  50%|█████     | 3/6 [00:38<00:37, 12.63s/it]

Team Members:  67%|██████▋   | 4/6 [00:52<00:26, 13.37s/it]

Team Members:  83%|████████▎ | 5/6 [01:12<00:15, 15.80s/it]

Team Members: 100%|██████████| 6/6 [01:46<00:00, 17.78s/it]

Rounds (+ Summary Round

Input token count: 225,758
Output token count: 15,268
Max token length: 20,428
Cost: $1.36


In [9]:
# Select preferred drug discovery approach
discussion_paths.append(save_dir / "drug_discovery" / "discussion_1.json")

In [10]:
# Load summaries
summaries = load_summaries(discussion_paths=discussion_paths)